In [1]:

import numpy as np
import timeit
import argparse
import cv2
import matplotlib.pyplot as plt
import math
import queue

In [2]:
class Node():
    def __init__(self, state, parent, move, cost): 

        self.state = state
        self.parent = parent
        self.move = move
        self.cost = cost
        # self.parent_state = parent_state
        
    def getState(self):
        return self.state
		
    def getParent(self):
        return self.parent

    def getParentState(self):
        if self.getParent() is None:
            return None
        return self.getParent().getState()
		
    def getMove(self):
	    return self.move
		
    def getCost(self):
        return self.cost

    def getFullPath(self):
        
        moves = []
        nodes = []
        current_node = self
        while(current_node.getMove() is not None):

            moves.append(current_node.getMove())
            nodes.append(current_node)
            current_node = current_node.getParent()

        nodes.append(current_node)
        moves.reverse()
        nodes.reverse()
        
        return moves, nodes

    def printStats(self):
        pass

In [3]:
def find_moves(current_node):
    i, j = current_node.getState()
    moves = ['N','NE', 'E', 'SE', 'S', 'SW','W', 'NW']
    final_moves = ['N','NE', 'E', 'SE', 'S', 'SW','W', 'NW']
    move_i = [i, i+1, i+1, i+1, i, i-1, i-1, i-1]
    move_j = [j+1, j+1, j, j-1, j-1, j-1, j, j+1]
    for move in range(len(moves)):
        if (isInObstacleSpace(move_i[move], move_j[move]) or current_node.getParentState() == [move_i[move], move_j[move]]):
            final_moves.remove(moves[move])
    print(final_moves)
    return final_moves

In [4]:
def isInObstacleSpace(i,j):

    if (i > 399 or i < 0 or j < 0 or j > 299):
        print('Tending out of boundary ; avoid')
        return 1

    #condition for cicle
    circle = (i-90)**2 + (j-70)**2
    if circle <= 1225:
        print('Tending towards circle ; avoid')
        return 1

    #condition for ellipse
    ellipse = ((i-246)**2)/(60*60) + ((j-145)**2)/(30*30)
    if ellipse <= 1.0:
        print('Tending towards ellipse ; avoid')
        return 1

    #condition for rectangle
    d1 = abs((j - 0.7002*i - 74.39) / (1 + (0.7002)**2)**(0.5))
    d2 = abs((j - 0.7002*i - 98.8) / (1 + (0.7002)**2)**(0.5))
    d3 = abs((j + 1.428*i - 176.55) / (1 + (1.428)**2)**(0.5))
    d4 = abs((j + 1.428*i - 439.44) / (1 + (1.428)**2)**(0.5))
    if (d1+d2 <= 22.0 and d3+d4 <= 152.0):
        print('Tending towards rectangle ; avoid')
        return 1

    #condition for C shaped object
    if ((i-200 >= 0 and 280-i >=0 and (j >= 230 and j <= 280)) and
        ((j-230 >= 0 or 280-j <=0) and (i >= 200 and i <= 230)) and
        not (i-210 >=0 and i-230<=0 and j>=240 and j<=270)):
        print('Tending towards C shaped object; avoid')
        return 1
    
    return 0


In [5]:
node_array = np.array([[Node([i,j], None, None, math.inf) for j in range(300)] for i in range(400)])

In [30]:
start_point = [2,2]
goal_point = [5,5]
nodes = queue.PriorityQueue()
init_node = Node(start_point, None, None, 0)

nodes.put((init_node.getCost(), init_node))

root2 = np.sqrt(2)

goal_reached = False

moves_cost = {'N':1, 'NE':root2, 'E':1, 'SE':root2, 'S':1, 'SW':root2, 'W':1, 'NW':root2}


In [31]:
while (not nodes.empty()):
    current_node = nodes.get()[1]
    i, j = current_node.getState()

    #define moves list
    moves_i = {'N':i, 'NE':i+1, 'E':i+1, 'SE':i+1, 'S':i, 'SW':i-1, 'W':i-1, 'NW':i-1}
    moves_j = {'N':j+1, 'NE':j+1, 'E':j, 'SE':j-1, 'S':j-1, 'SW':j-1, 'W':j, 'NW':j+1}

    if (current_node.getState() == goal_point):
        print('Goal reached')
        print("Total number of nodes explored:", len(visited_states))
        print("The cost of path: ", current_node.getCost())
        full_path, node_path = current_node.getFullPath()
        goal_reached = True
        break
    else:
        # find the moves from the current position
        moves = find_moves(current_node)
        parent_cost = current_node.getCost()
        # iterate through each move and find corresponding child
        for move in moves:
            child_state = [moves_i.get(move), moves_j.get(move)]
            new_cost = parent_cost + moves_cost.get(move)
            # if not visited
            if (node_array[child_state[0], child_state[1]].getCost() == math.inf):
                child_node = Node(child_state, current_node, move, new_cost)
                node_array[child_state[0], child_state[1]] = child_node
                nodes.put((child_node.getCost(), child_node))
            else :
                if (new_cost < node_array[child_state[0], child_state[1]].getCost()):
                    child_node = Node(child_state, current_node, move, new_cost)
                    node_array[child_state[0], child_state[1]] = child_node
    
    if (goal_reached): break
    

['N', 'NE', 'E', 'SE', 'S', 'SW', 'W', 'NW']
['N', 'NE', 'E', 'S', 'SW', 'W', 'NW']


TypeError: '<' not supported between instances of 'Node' and 'Node'

In [18]:
current_node = nodes.get()[1]

In [19]:
nodes.empty()

True